<a href="https://colab.research.google.com/github/akatwinkle/bab_bab/blob/main/%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = 'http://openapi.seoul.go.kr:8088/sample/json/energyUseDataSummaryInfo/1/5/2015/01'
url


'http://openapi.seoul.go.kr:8088/sample/json/energyUseDataSummaryInfo/1/5/2015/01'

In [1]:
import requests

url = 'https://data.seoul.go.kr/dataList/OA-15361/S/1/datasetView.do'
api_key = '4a48554755796f6f36375856727974'

params = {
    "MM_TYPE": "개인"
}

resp = requests.get(url, params = params)

j = resp.json()

print(j)

JSONDecodeError: Expecting value: line 23 column 1 (char 57)